In [1]:
import os
import csv
import time
import pandas as pd
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor    # Concurrent execution using threads
from openai import OpenAI

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [3]:
# provide dataset name among ai2_arc, gsm8k, lukaemon/mmlu
dataset_name = "ai2_arc"

In [4]:
if(dataset_name == "ai2_arc"):
    possible_configs = [
    'ARC-Challenge',
    'ARC-Easy'
    ]
    # columns to translate
    columns = ['question','choices']
    # columns not to translate, to keep in converted dataset as is.
    columns_asis = ['id','answerKey']

elif(dataset_name == "gsm8k"):
    possible_configs = [
    "main",
    "socratic"
    ]
    # columns to translate
    columns = ['question','answer']
    # columns not to translate, to keep in converted dataset as is.
    columns_asis = []
    
elif(dataset_name == "lukaemon/mmlu"):
    possible_configs = [
    "high_school_european_history",
    "business_ethics",
    "clinical_knowledge",
    "medical_genetics",
    "high_school_us_history",
    # "high_school_physics",
    # "high_school_world_history",
    # "virology",
    # "high_school_microeconomics",
    # "econometrics",
    # "college_computer_science",
    # "high_school_biology",
    # "abstract_algebra",
    # "professional_accounting",
    # "philosophy",
    # "professional_medicine",
    # "nutrition",
    # "global_facts",
    # "machine_learning",
    # "security_studies",
    # "public_relations",
    # "professional_psychology",
    # "prehistory",
    # "anatomy",
    # "human_sexuality",
    # "college_medicine",
    # "high_school_government_and_politics",
    # "college_chemistry",
    # "logical_fallacies",
    # "high_school_geography",
    # "elementary_mathematics",
    # "human_aging",
    # "college_mathematics",
    # "high_school_psychology",
    # "formal_logic",
    # "high_school_statistics",
    # "international_law",
    # "high_school_mathematics",
    # "high_school_computer_science",
    # "conceptual_physics",
    # "miscellaneous",
    # "high_school_chemistry",
    # "marketing",
    # "professional_law",
    # "management",
    # "college_physics",
    # "jurisprudence",
    # "world_religions",
    # "sociology",
    # "us_foreign_policy",
    # "high_school_macroeconomics",
    # "computer_security",
    # "moral_scenarios",
    # "moral_disputes",
    # "electrical_engineering",
    # "astronomy",
    # "college_biology",
    ]
    # columns to translate
    columns = ['input','A','B','C','D']
    # columns not to translate, to keep in converted dataset as is.
    columns_asis = ['target']

In [5]:
dataset = []
if(dataset_name == "ai2_arc"):
    for config in possible_configs:
        base_url = f"https://huggingface.co/api/datasets/allenai/ai2_arc/parquet/{config}"
        data_files = {"train": base_url + "/train/0.parquet","test":base_url + "/test/0.parquet", "validation": base_url + "/validation/0.parquet"}
        dataset_slice = load_dataset("parquet", data_files=data_files)
        dataset.append(dataset_slice)
elif(dataset_name == "gsm8k"):
    for config in possible_configs:
        base_url = f"https://huggingface.co/api/datasets/gsm8k/parquet/{config}"
        data_files = {"train": base_url + "/train/0.parquet","test":base_url + "/test/0.parquet"}
        dataset_slice = load_dataset("parquet", data_files=data_files)
        dataset.append(dataset_slice)
elif(dataset_name == "lukaemon/mmlu"):
    for config in possible_configs:
        dataset_slice = load_dataset(dataset_name, config,ignore_verifications=True)
        dataset.append(dataset_slice)

Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/default-94e720c37a295035/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/default-648555a0f98e748d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
client = OpenAI()
def translate_gpt(text: str):
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "translate the given text in english to hinglish.(written in english alphabet but sounding like reading hindi language)"},
            {"role": "system", "content": "if input is 'how are you' return 'kaise ho ap'"},
            # {"role": "system", "content": "if input is 'Add a new weekly reminder for Sunday Brunch at 9 : 30 am' return '9 : 30 am ko Sunday Brunch ke liye ek naya weekly reminder add karen'"},
            # {"role": "system", "content": "if input is 'Using the timer to find out how long this 5K race will take me' return '5k ki daud me mujhe kitna samay lagega ye jaanne ke liye timer ka use karo'"},
            # {"role": "system", "content": "if input is 'how long will it take to get to richmond' return 'richmond tak jane ke liye kitni der lagegi'"},
            {"role": "system", "content": "suppose you are a native hindi speaker who also speaks english, but since most phones offer english keyboard by default you choose to write hindi in english letters. return the input english texts in hinglish"},
            {"role": "system", "content": "if input is '{ 'text': [ 'cooling of flowing magma.', 'converging of crustal plates.', 'deposition of river sediments.', 'solution of carbonate minerals.' ], 'label': [ 'A', 'B', 'C', 'D' ] }' return '{ 'text': [ 'flowing magma ka thnda hone.', 'crustal plates ka converge hona.', 'river sediments ka deposition hona.', 'carbonate minerals ka solution.' ], 'label': [ 'A', 'B', 'C', 'D' ] }'"},
            {"role": "system", "content": "if input is 'The end result in the process of photosynthesis is the production of sugar and oxygen. Which step signals the beginning of photosynthesis' return 'photosynthesis process ke last m sugar aur oxygen produce hota h, kis step se photosynthesis ke start hone ka pta chlta h'"},  
            {"role": "system", "content": "numbers, nouns, verbs stay in English and rest is in Hindi. Do not give original text back"},
            {"role": "system", "content": "do not translate to hindi, translate to hinglish"},
            {"role": "user", "content": text}
        ]
    )
    return completion.choices[0].message.content

In [7]:
translate_gpt('{ "text": [ "cooling of flowing magma.", "converging of crustal plates.", "deposition of river sediments.", "solution of carbonate minerals." ], "label": [ "A", "B", "C", "D" ] }')

'{ "text": [ "flowing magma ka thanda hona.", "crustal plates ka converge hona.", "river sediments ka deposition hona.", "carbonate minerals ka solution hona." ], "label": [ "A", "B", "C", "D" ] }'

In [8]:
translate_gpt('The end result in the process of photosynthesis is the production of sugar and oxygen. Which step signals the beginning of photosynthesis?')

'The end result in the process of photosynthesis is the production of sugar aur oxygen. Kaunsa step photosynthesis ka start hone ka signal deta hai?'

In [9]:
translate_gpt('how are you')

'kaise ho ap'

In [10]:
%%time

if(dataset_name=='lukaemon/mmlu'):
    os.makedirs("lukaemon_mmlu_files", exist_ok=True)
    os.chdir("lukaemon_mmlu_files")
else:
    os.makedirs(dataset_name + "_files", exist_ok=True)
    os.chdir(dataset_name + "_files")

for i in range(len(possible_configs)):
    for set in dataset[i]:
        set_list = []
        
        for col in columns:
            values = [str(item[col]) for item in dataset[i][set]]
            
            # Use ThreadPoolExecutor for parallel translation
            if __name__ == '__main__':
                result =[]
                with ThreadPoolExecutor(max_workers=16) as exe:
                    # Maps the method with a list of values.
                    # result = list(exe.map(translate_gpt,values[:1]))
                    
                    batch_size = 1
                    for j in range(0,len(values[:3]),batch_size):
                        j_end = min(j + batch_size, len(values[:1]))
                        res = list(exe.map(translate_gpt,values[j:j_end]))
                        # time.sleep(2)
                        result.extend(res)
                    
            set_list.append(result)

        # Create folders for each configuration
        current_directory = os.getcwd()
        
        # Specify the path of the 'config' folder
        config_folder_path = os.path.join(current_directory, possible_configs[i])
        
        # Create the 'config' folder
        os.makedirs(config_folder_path, exist_ok=True) 
    
        # Transpose the 2D list
        transposed_data = list(map(list, zip(*set_list)))
        
        # to add untranslated columns in dataset
        for row in range(len(transposed_data)):
            for col in columns_asis:
                if col=='id':
                        position = 0
                else:
                    position = len(transposed_data[row])
                transposed_data[row].insert(position, dataset[i][set][col][row]) 
        
            
        path = os.path.join(possible_configs[i], f'{set}.csv')

        # append to previosly created csv file in case full dataset was not converted
        with open(path, 'a', encoding='utf-8') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            # write.writerow(columns)
            write.writerows(transposed_data)


CPU times: total: 594 ms
Wall time: 19.1 s


In [11]:
df = pd.read_csv(path, header=None)

# Print the DataFrame
df.head()

ARC-Easy\validation.csv


,0,1,2,3
0,MCAS_2000_4_6,kaunsi technology sabse haal hi mein develop h...,"{'text': ['cellular telephone', 'television', ...",A
